In [4]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import pickle
import tensorflow.keras
import yaml


import importlib

import Model10_OOP
importlib.reload(Model10_OOP)
from Model10_OOP import ConvModel

import BaseData20Fold
importlib.reload(BaseData20Fold)
from BaseData20Fold import DataProcessor

import Plots
importlib.reload(Plots)
from Plots import plot_CM

# COntains 2stream Neural Network to process fine fatures and coarse features separate


In [5]:
cwd = Path.cwd()
cfg = 'config_mdl10.yaml'

with open(cfg, 'r') as file:
    config = yaml.safe_load(file)
seeds = config['seeds']

folds = np.arange(1,21)

datapath = cwd / '..' / 'Data' / 'DHG2016' / 'owpg_dcnn'

%load_ext tensorboard
%load_ext version_information
#!rm -rf ./logs/

logsdir = cwd / '..' / 'logs'
# !rmdir /s /q {logsdir}
#!tensorboard --logdir {logsdir} --host localhost --port 6006


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information


In [ ]:
px.histogram(x=data_processor.Y_test, title='Train Labels').show()

In [6]:
resultsDFlong = pd.DataFrame()
folder = 'owpg_dcnn'
data_processor = DataProcessor(cfg)
data_processor.load_handgestdata()
data_processor.handangles2windows()
data_processor.save_config(folder)
data_processor.save_windowsets(folder)

#folds = [5]
    
for fold in folds:
    data_processor = DataProcessor(cfg, fold = fold)
    # data_processor.load_handgestdata()
    # data_processor.handangles2windows()
    # data_processor.save_windowsets(folder)
    data_processor.load_windows(folder)
    data_processor.processwindows()
    data_processor.save_windowsets_processed(folder, name='Fold'+str(fold))


permuting Samples using seed 26
Make Windows and apply framreferences...
Processing subject 1
Processing subject 2
Processing subject 3
Processing subject 4
Processing subject 5
Processing subject 6
Processing subject 7
Processing subject 8
Processing subject 9
Processing subject 10
Processing subject 11
Processing subject 12
Processing subject 13
Processing subject 14
Processing subject 15
Processing subject 16
Processing subject 17
Processing subject 18
Processing subject 19
Processing subject 20
Using seed 26
Fold: 1
Standardize and padding of data...
Standardize data...train
Write data into windows...
Standardize data...valid
Write data into windows...
Using seed 26
Fold: 2
Standardize and padding of data...
Standardize data...train
Write data into windows...
Standardize data...valid
Write data into windows...
Using seed 26
Fold: 3
Standardize and padding of data...
Standardize data...train
Write data into windows...
Standardize data...valid
Write data into windows...
Using seed 26

In [11]:
datapath = cwd / '..' / 'Data' / 'DHG2016' / 'owpg_dcnn'

results_valid_labels_list = []
results_scores_list_valid = []
results_scores_list_train = []

for seedidx, n_fold in enumerate(folds):
    print('evaluating Fold', n_fold)
    seed_val = seeds[seedidx]

    fold = pickle.load(open((datapath / ('Fold' + str(n_fold) + '.pkl')).resolve(), 'rb'))
    Y_train_oh = fold['train']['Y_oh']
    Y_valid_oh = fold['valid']['Y_oh']
    X_train = fold['train']['X']
    X_valid = fold['valid']['X']


    like7 = ConvModel(cfg, 'Test', seed = seed_val)
    like7.compile_model()

    # Train the model
    like7.train_model(X_train, Y_train_oh, X_valid, Y_valid_oh)

    #check size of Y_oh to see if 14 and 15 class differentiation is necessary
    # check numbe rof datasets to see if sucessfully just used one window per gesture
    Y_prob_train = like7.model.predict(X_train)
    Y_prob_valid = like7.model.predict(X_valid)
    Y_score_true = np.max(Y_prob_valid, axis=-1)

    Y_hat_train = np.argmax(Y_prob_train, axis=-1) + 1 # +1 ino rder to have the same labels fo rclasses 1-14 as with a 15 class problems
    Y_hat_valid = np.argmax(Y_prob_valid, axis=-1) + 1

    results_train = like7.model.evaluate(X_train, Y_train_oh, verbose = 0, return_dict=True)
    results_valid = like7.model.evaluate(X_valid, Y_valid_oh, verbose = 0, return_dict=True)

    print(f'NLL: {results_valid['NLL']:,.2f}')
    #print(f'Validation score: {results['loss']:,.2f}')
    print(f'Validation accuracy: {results_valid['accuracy']:,.2f}')
    print(f'Validation F1Score: {results_valid['f1_score']:,.2f}')

    results_valid_labels_list.append(pd.DataFrame({'Fold': n_fold, 'Y': fold['valid']['Y'], 'Y_hat': Y_hat_valid, 'Y_prob': Y_score_true}))
    results_scores_list_train.append(pd.DataFrame(results_train, index = [n_fold]))
    results_scores_list_valid.append(pd.DataFrame(results_valid, index = [n_fold]))
    #results_scores_list.append({'Fold': n_fold, 'Accuracy_train': accuracy_train, 'Accuracy_valid': accuracy_valid, 'F1-Score_train': f1_score_train, 'F1-Score_valid': f1_score_valid, 'NLL_train': nll_train, 'NLL_valid': nll_valid})
    
results_scores = pd.concat([pd.concat(results_scores_list_train).add_suffix('_train'),pd.concat(results_scores_list_valid).add_suffix('_valid')],axis = 1)
results_scores.rename(columns = {'fold_train': 'fold'}, inplace = True)
results_valid_labels = pd.concat(results_valid_labels_list)


evaluating Fold 1
Setting seed to 5
Epoch 1/20
42/42 - 30s - 707ms/step - NLL: 1.8382 - accuracy: 0.3992 - f1_score: 0.3915 - loss: 2.3304 - precision: 0.7905 - recall: 0.1744 - val_NLL: 1.5124 - val_accuracy: 0.5786 - val_f1_score: 0.5749 - val_loss: 1.9466 - val_precision: 0.8205 - val_recall: 0.2286
Epoch 2/20
42/42 - 8s - 190ms/step - NLL: 1.0293 - accuracy: 0.6744 - f1_score: 0.6721 - loss: 1.4223 - precision: 0.8369 - recall: 0.5094 - val_NLL: 1.1356 - val_accuracy: 0.6643 - val_f1_score: 0.6545 - val_loss: 1.5529 - val_precision: 0.8333 - val_recall: 0.4643
Epoch 3/20
42/42 - 8s - 189ms/step - NLL: 0.7755 - accuracy: 0.7462 - f1_score: 0.7461 - loss: 1.1429 - precision: 0.8632 - recall: 0.6429 - val_NLL: 1.0379 - val_accuracy: 0.6500 - val_f1_score: 0.6417 - val_loss: 1.3811 - val_precision: 0.7981 - val_recall: 0.5929
Epoch 4/20
42/42 - 8s - 186ms/step - NLL: 0.6635 - accuracy: 0.7861 - f1_score: 0.7854 - loss: 1.0175 - precision: 0.8769 - recall: 0.7068 - val_NLL: 0.9230 - val

In [13]:
results_valid_labels.to_pickle(datapath/'results_valid_labels_dcnn.pkl')
results_scores.to_pickle(datapath/'results_scores_dcnn.pkl')

In [14]:
mean = results_scores.mean()
std = results_scores.std()

print(f'accuracy on training data - frames: {mean["accuracy_train"]*100:.1f}% +-  {std["accuracy_train"]*100:.1f}%')
print(f'F1_Score on training data - frames: {mean["f1_score_train"]*100:.1f}% +- {std["f1_score_train"]*100:.1f}%')
print(f'NLL on training data - frames: {mean["NLL_train"]:.2f} +- {std["NLL_train"]:.2f}')

print(f'accuracy on validation data - frames: {mean['accuracy_valid']*100:.1f}% +- {std['accuracy_valid']*100:.1f}%')
print(f'F1-Score on validation data - frames: {mean['f1_score_valid']*100:.1f}% +- {std['f1_score_valid']*100:.1f}%')
print(f'NLL on validation data - frames: {mean['NLL_valid']:.2f} +- {std['NLL_valid']:.2f}')

accuracy on training data - frames: 93.0% +-  3.3%
F1_Score on training data - frames: 92.9% +- 3.3%
NLL on training data - frames: 0.23 +- 0.10
accuracy on validation data - frames: 78.5% +- 8.0%
F1-Score on validation data - frames: 77.6% +- 8.2%
NLL on validation data - frames: 0.68 +- 0.31



Ab hier optionaler Code




In [6]:
resultsDFlong

,padding,fold,seed,NLL,accuracy,f1_score
0,zeros,1,26,1.085798,0.692857,0.668352
1,zeros,1,1027,1.104139,0.650000,0.645547
2,zeros,2,26,0.685636,0.764286,0.768051
3,zeros,2,1027,0.750157,0.764286,0.764804
4,zeros,3,26,1.217961,0.642857,0.633530
5,zeros,3,1027,1.166725,0.592857,0.583506
6,zeros,4,26,0.833712,0.757143,0.757152
7,zeros,4,1027,0.792932,0.757143,0.753700
8,zeros,5,26,0.914476,0.775000,0.763380
9,zeros,5,1027,1.090410,0.725000,0.719685


In [28]:
# valid padding and 200 outputneurons dilation factor 3
print(resultsDFlong.groupby(['padding']).mean())
print(resultsDFlong.groupby(['padding']).std())

         fold          seed       NLL  accuracy  f1_score
padding                                                  
zeros     4.5  2.292545e+08  0.757024  0.784375  0.780361
            fold          seed       NLL  accuracy  f1_score
padding                                                     
zeros    2.44949  5.661250e+08  0.282956  0.088701  0.092379


In [25]:
# valid padding and 200 outputneurons
print(resultsDFlong.groupby(['padding']).mean())
print(resultsDFlong.groupby(['padding']).std())

         fold          seed       NLL  accuracy  f1_score
padding                                                  
zeros     4.5  2.292545e+08  0.756668  0.780357  0.778716
            fold          seed       NLL  accuracy  f1_score
padding                                                     
zeros    2.44949  5.661250e+08  0.268416  0.079057  0.081608


In [21]:
# causal padding and 200 outputneurons
print(resultsDFlong.groupby(['padding']).mean())
print(resultsDFlong.groupby(['padding']).std())

         fold          seed       NLL  accuracy  f1_score
padding                                                  
zeros     4.5  2.292545e+08  0.759772  0.780804  0.778895
            fold          seed       NLL  accuracy  f1_score
padding                                                     
zeros    2.44949  5.661250e+08  0.257969  0.086391  0.089062


In [18]:
# causal padding
print(resultsDFlong.groupby(['padding']).mean())
print(resultsDFlong.groupby(['padding']).std())

         fold          seed       NLL  accuracy  f1_score
padding                                                  
zeros     4.5  2.292545e+08  0.758842  0.777232  0.775612
            fold          seed      NLL  accuracy  f1_score
padding                                                    
zeros    2.44949  5.661250e+08  0.28643  0.089295  0.092065


In [17]:
print(resultsDFlong.groupby(['padding']).mean())
print(resultsDFlong.groupby(['padding']).std())

         fold          seed       NLL  accuracy  f1_score
padding                                                  
zeros     4.5  2.292545e+08  0.722857  0.787946  0.785496
            fold          seed       NLL  accuracy  f1_score
padding                                                     
zeros    2.44949  5.661250e+08  0.287925  0.094355  0.101989


In [14]:
# 100 neuronen im klassifikator
print(resultsDFlong.groupby(['padding']).mean())
print(resultsDFlong.groupby(['padding']).std())

         fold          seed      NLL  accuracy  f1_score
padding                                                 
zeros     4.5  2.292545e+08  0.75984  0.775893  0.772963
            fold          seed       NLL  accuracy  f1_score
padding                                                     
zeros    2.44949  5.661250e+08  0.304002  0.085836  0.089565


In [8]:
# nur 1 maxpooling layer

print(resultsDFlong.groupby(['padding']).mean())
print(resultsDFlong.groupby(['padding']).std())

         fold          seed       NLL  accuracy  f1_score
padding                                                  
zeros     4.5  2.292545e+08  0.756668  0.780357  0.778716
            fold          seed       NLL  accuracy  f1_score
padding                                                     
zeros    2.44949  5.661250e+08  0.268416  0.079057  0.081608


In [52]:
print(resultsDFlong.groupby(['padding']).mean())
print(resultsDFlong.groupby(['padding']).std())

         fold          seed       NLL  accuracy  f1_score
padding                                                  
zeros     4.5  2.292545e+08  0.707225  0.801339   0.79808
            fold          seed       NLL  accuracy  f1_score
padding                                                     
zeros    2.44949  5.661250e+08  0.287982  0.085798   0.08959


In [42]:
# firstframe reference
print(resultsDFlong.groupby(['padding', 'seed']).mean())
print(resultsDFlong.groupby(['padding', 'seed']).std())

              fold       NLL  accuracy  f1_score
padding seed                                    
zeros   26     4.5  0.763702  0.784375  0.782688
        1027   4.5  0.726717  0.788839  0.787662
                 fold       NLL  accuracy  f1_score
padding seed                                       
zeros   26    2.44949  0.317027  0.086243  0.087903
        1027  2.44949  0.254232  0.065282  0.065067


In [7]:
# witohout quaternion standardization

print(resultsDFlong.groupby(['padding', 'seed']).mean())
print(resultsDFlong.groupby(['padding', 'seed']).std())

              fold       NLL  accuracy  f1_score
padding seed                                    
zeros   26     4.5  1.082803  0.692857  0.685068
        1027   4.5  1.196937  0.658036  0.654548
                 fold       NLL  accuracy  f1_score
padding seed                                       
zeros   26    2.44949  0.293481  0.073615  0.073758
        1027  2.44949  0.258127  0.070239  0.068022


In [7]:
# standardize quaternion as well

print(resultsDFlong.groupby(['padding', 'seed']).mean())
print(resultsDFlong.groupby(['padding', 'seed']).std())

              fold       NLL  accuracy  f1_score
padding seed                                    
zeros   26     4.5  1.002874  0.698661  0.690919
        1027   4.5  1.119779  0.656250  0.649268
                 fold       NLL  accuracy  f1_score
padding seed                                       
zeros   26    2.44949  0.242467  0.065603  0.068252
        1027  2.44949  0.281625  0.086555  0.093948


In [ ]:
%version_information tensorflow, numpy, matplotlib, plotly, pandas, keras